# Token classification improving via GAN

In [1]:
%load_ext nb_black
%load_ext autoreload
%autoreload 2

<IPython.core.display.Javascript object>

In [5]:
%cd gan-plus-nlp-main/

/home/valperovich/projects/other/std/gan-plus-nlp-main


<IPython.core.display.Javascript object>

In [2]:
# !pip install neptune-client
import os
import gc
import sys
import json
import numpy as np
import pandas as pd
import importlib as imp
import neptune as neptune
from tqdm import tqdm, tqdm_notebook
from sklearn.manifold import TSNE
from sklearn.metrics import classification_report, f1_score
import seaborn as sns
import matplotlib.pyplot as plt
from typing import List, Dict
import warnings

import torch
import torch.nn.functional as F

warnings.simplefilter("ignore")
sys.path.append("..")
# sys.path.append('gan-text-classification')
secret = json.load(open("secret.json"))

<IPython.core.display.Javascript object>

# Data Loading

In [9]:
from datasets import load_dataset

dataset_name = (
    "tner/ontonotes5"  # 'conll2003' wnut_17 tner/tweebank_ner tner/ontonotes5
)
ner_dataset = load_dataset(dataset_name)
ner_dataset

Found cached dataset ontonotes5 (/home/valperovich/.cache/huggingface/datasets/tner___ontonotes5/ontonotes5/1.0.0/58d8410f24e689c113094eef1d1686365ba9155c66b57bdf8fa4273307c37612)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 59924
    })
    validation: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 8528
    })
    test: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 8262
    })
})

<IPython.core.display.Javascript object>

In [15]:
# _type = "validation"
# _indexes = np.random.permutation(len(ner_dataset[_type]))
# ner_dataset[_type] = ner_dataset[_type].select(_indexes[:2500])

<IPython.core.display.Javascript object>

In [10]:
ner_dataset_df = ner_dataset["train"].to_pandas()
BIO_TAGS = {
    "O": 0,
    "B-CARDINAL": 1,
    "B-DATE": 2,
    "I-DATE": 3,
    "B-PERSON": 4,
    "I-PERSON": 5,
    "B-NORP": 6,
    "B-GPE": 7,
    "I-GPE": 8,
    "B-LAW": 9,
    "I-LAW": 10,
    "B-ORG": 11,
    "I-ORG": 12,
    "B-PERCENT": 13,
    "I-PERCENT": 14,
    "B-ORDINAL": 15,
    "B-MONEY": 16,
    "I-MONEY": 17,
    "B-WORK_OF_ART": 18,
    "I-WORK_OF_ART": 19,
    "B-FAC": 20,
    "B-TIME": 21,
    "I-CARDINAL": 22,
    "B-LOC": 23,
    "B-QUANTITY": 24,
    "I-QUANTITY": 25,
    "I-NORP": 26,
    "I-LOC": 27,
    "B-PRODUCT": 28,
    "I-TIME": 29,
    "B-EVENT": 30,
    "I-EVENT": 31,
    "I-FAC": 32,
    "B-LANGUAGE": 33,
    "I-PRODUCT": 34,
    "I-ORDINAL": 35,
    "I-LANGUAGE": 36,
}
BIO_TAGS = list(BIO_TAGS)
# BIO_TAGS

<IPython.core.display.Javascript object>

In [3]:
# tag_col = "tags"  # "ner_tags" tags
# # BIO_TAGS = ner_dataset["train"].features[tag_col].feature.names
# get_ids2tag = lambda ids: [BIO_TAGS[t] for t in ids]
# NUM_LABELS = len(
#     BIO_TAGS
# )  # ner_dataset["train"].features["ner_tags"].feature.num_classes


# def add_ner_labels(examples):
#     examples["ner_labels"] = [get_ids2tag(x) for x in examples[tag_col]]
#     return examples


# ner_dataset = ner_dataset.map(add_ner_labels, batched=True)

# BIO_TAGS, NUM_LABELS

<IPython.core.display.Javascript object>

In [17]:
ner_dataset_df = ner_dataset["test"].to_pandas()
ner_dataset_df.sample(3)

,tokens,tags,ner_labels
6545,"[Furthermore, ,, the, initial, results, of, jo...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 11, 12, 0, 11, 12,...","[O, O, O, O, O, O, O, O, O, B-ORG, I-ORG, O, B..."
7168,"[Now, she, had, a, job, with, an, architectura...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
7730,"[Why, ?]","[0, 0]","[O, O]"


<IPython.core.display.Javascript object>

In [18]:
ner_dataset_df["tokens"].apply(len).describe()

count    8262.000000
mean       18.484992
std        13.702388
min         1.000000
25%         8.000000
50%        16.000000
75%        25.000000
max       151.000000
Name: tokens, dtype: float64

<IPython.core.display.Javascript object>

In [19]:
# from spacy import displacy
# import spacy
# nlp = spacy.blank('en')

# def ner_display(row):
#     sent = ' '.join(row['tokens'])
#     doc = nlp(sent)
#     print(doc.ents)
#     ents = []
#     prev_i = -1
#     prev_j = -2
#     for label, token in zip(row['ner_labels'], row['tokens']):
#         if label != "O":
#             i, j = sent.find(token), sent.find(token)+len(token)
#             if i == (prev_j + 1):
#                 ents.pop()
#                 ent = doc.char_span(prev_i, j, label=label.split('-')[-1])
#                 prev_j = j
#             else:
#                 ent = doc.char_span(i, j, label=label.split('-')[-1])
#                 prev_i, prev_j = i,j
#             ents.append(ent)
#     print(ents) 
#     doc.ents = ents
#     return displacy.render(doc, style="ent", jupyter=True, options={'distance': 90})


<IPython.core.display.Javascript object>

In [20]:
# row = ner_dataset_df[ner_dataset_df['ner_labels'].apply(set).apply(len) > 5].sample().iloc[0]
# print("\n\n")
# ner_display(row)
# print("\n\n")

<IPython.core.display.Javascript object>

In [21]:
# ner_dataset_df["tokens"].apply(len).describe()

<IPython.core.display.Javascript object>

In [22]:
def get_entity_names(biotags: List[str]) -> List[str]:
    entity_names = set([])
    for i, x in enumerate(biotags):
        if len(x) > 1:
            entity_names.add(x[2:])
    return entity_names


ENTITY_NAMES = get_entity_names(BIO_TAGS)
ENTITY_NAMES

{'CARDINAL',
 'DATE',
 'EVENT',
 'FAC',
 'GPE',
 'LANGUAGE',
 'LAW',
 'LOC',
 'MONEY',
 'NORP',
 'ORDINAL',
 'ORG',
 'PERCENT',
 'PERSON',
 'PRODUCT',
 'QUANTITY',
 'TIME',
 'WORK_OF_ART'}

<IPython.core.display.Javascript object>

In [23]:
def get_tags_value_counts(dataset, tags, feature="ner_labels"):
    if feature not in ner_dataset["train"].features.keys():
        raise AssertionError(f"Dataset has no {feature}")
    id2tag = {}
    tag2count = {}
    for sample in dataset:
        for bio_tag in sample[feature]:
            if bio_tag[0] == "I":
                continue
            tag_name = bio_tag[2:]
            if tag_name in tags:
                tag2count[tag_name] = tag2count.get(tag_name, 0) + 1
    return tag2count


TRAIN_NER_COUNTS = get_tags_value_counts(ner_dataset["train"], ENTITY_NAMES)
VALID_NER_COUNTS = get_tags_value_counts(ner_dataset["validation"], ENTITY_NAMES)
TEST_NER_COUNTS = get_tags_value_counts(ner_dataset["test"], ENTITY_NAMES)
print("TRAIN", TRAIN_NER_COUNTS)
print("VALID", VALID_NER_COUNTS)
print("TEST", TEST_NER_COUNTS)

TRAIN {'CARDINAL': 7355, 'DATE': 10922, 'PERSON': 15429, 'NORP': 6870, 'GPE': 15405, 'LAW': 282, 'ORG': 12820, 'PERCENT': 1763, 'ORDINAL': 1640, 'MONEY': 2411, 'WORK_OF_ART': 974, 'FAC': 860, 'TIME': 1233, 'LOC': 1514, 'QUANTITY': 657, 'PRODUCT': 606, 'EVENT': 748, 'LANGUAGE': 304}
VALID {'GPE': 620, 'CARDINAL': 274, 'ORDINAL': 56, 'EVENT': 39, 'LOC': 51, 'ORG': 501, 'PERSON': 583, 'NORP': 258, 'DATE': 451, 'PRODUCT': 18, 'MONEY': 73, 'LANGUAGE': 7, 'LAW': 9, 'WORK_OF_ART': 41, 'TIME': 68, 'QUANTITY': 25, 'PERCENT': 59, 'FAC': 41}
TEST {'DATE': 1602, 'GPE': 2240, 'ORG': 1795, 'MONEY': 313, 'PERCENT': 349, 'CARDINAL': 935, 'TIME': 212, 'ORDINAL': 195, 'LOC': 179, 'QUANTITY': 105, 'NORP': 841, 'PRODUCT': 76, 'PERSON': 1988, 'LAW': 40, 'FAC': 135, 'WORK_OF_ART': 166, 'EVENT': 63, 'LANGUAGE': 22}


<IPython.core.display.Javascript object>

## Experiment

In [24]:
import torch
import torch.nn.functional as F

os.environ["CUDA_VISIBLE_DEVICES"] = "7"

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("There are %d GPU(s) available." % torch.cuda.device_count())
    print("We will use the GPU:", torch.cuda.get_device_name())
else:
    print("No GPU available, using the CPU instead.")
    device = torch.device("cpu")
print(torch.cuda.memory_allocated())

There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 2080 Ti
0


<IPython.core.display.Javascript object>

In [25]:
model_name = "distilbert-base-uncased"

<IPython.core.display.Javascript object>

In [27]:
import sys

try:
    del sys.modules["base"]
    del sys.modules["model"]
    del sys.modules["model.discriminator"]
    del sys.modules["model.generator"]
    del sys.modules["model.utils"]
    del sys.modules["trainer"]
except:
    print("pass")

torch.cuda.empty_cache()
gc.collect()

import model

model = imp.reload(model)

pass


<IPython.core.display.Javascript object>

In [28]:
CONFIG = dict(
    TASK="ner",
    encoder_name=model_name,
    frozen_backbone=False,
    batch_size=16,
    max_seq_length=32,
    noise_size=100,
    dataset_train_size=len(ner_dataset["train"]),
    dataset_valid_size=len(ner_dataset["validation"]),
    dataset_test_size=len(ner_dataset["test"]),
    num_labels=NUM_LABELS,
    label_names=BIO_TAGS,
    lr_discriminator=5e-5,
    lr_generator=5e-5,
    epsilon=1e-8,
    num_train_epochs=5,
    multi_gpu=False,
    dropout_rate=0.2,
    apply_scheduler=True,
    warmup_proportion_d=0.1,
    warmup_proportion_g=0.0,
    fake_label_index=-1,
    dataset=dataset_name,
    save_path="../weights/best_model.pth",
)

<IPython.core.display.Javascript object>

In [29]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True,
        max_length=CONFIG["max_seq_length"],
    )

    labels = []
    for i, label in enumerate(examples[tag_col]):
        word_ids = tokenized_inputs.word_ids(
            batch_index=i
        )  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif (
                word_idx != previous_word_idx
            ):  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

<IPython.core.display.Javascript object>

In [30]:
from copy import copy
from datasets import Dataset
from transformers import DataCollatorForTokenClassification
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

LABELED_SIZE = 200
UNLABELED_SIZE = 3000
FULL_SIZE = LABELED_SIZE + UNLABELED_SIZE
multiplier = int(np.log2(FULL_SIZE / LABELED_SIZE))
multiplier = max(1, multiplier)
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

print("Multiplier:", multiplier)

np.random.seed(42)


def prepare_experiment_datasets(LABELED_SIZE, UNLABELED_SIZE, FULL_SIZE, multiplier):
    tokenized_dataset = ner_dataset.map(tokenize_and_align_labels, batched=True)
    print(LABELED_SIZE, UNLABELED_SIZE, FULL_SIZE, multiplier)
    tokenized_train_df = tokenized_dataset["train"].to_pandas()
    tokenized_train_df_labeled = tokenized_train_df.sample(LABELED_SIZE)
    tokenized_train_df_labeled["labeled_mask"] = True

    tokenized_train_df = tokenized_train_df.sample(UNLABELED_SIZE)
    tokenized_train_df["labeled_mask"] = False
    tokenized_train_df["labels"] = tokenized_train_df["labels"].apply(
        lambda x: [-100] * len(x)
    )

    for _ in range(multiplier):
        tokenized_train_df = tokenized_train_df.append(tokenized_train_df_labeled)

    tokenized_dataset["train"] = Dataset.from_pandas(
        tokenized_train_df, preserve_index=False
    )
    tokenized_dataset["train_only_labeled"] = Dataset.from_pandas(
        tokenized_train_df_labeled, preserve_index=False
    )
    tokenized_dataset["test"] = tokenized_dataset["test"].add_column(
        "labeled_mask", [True] * len(tokenized_dataset["test"])
    )
    tokenized_dataset["validation"] = tokenized_dataset["validation"].add_column(
        "labeled_mask", [True] * len(tokenized_dataset["validation"])
    )
    tokenized_dataset = tokenized_dataset.select_columns(
        ["input_ids", "attention_mask", "labels", "labeled_mask"]
    )
    print(
        "TRAIN (FOR only discriminator):", len(tokenized_dataset["train_only_labeled"])
    )
    print("TRAIN (FOR GAN):", len(tokenized_dataset["train"]))
    return tokenized_dataset


tokenized_dataset = prepare_experiment_datasets(
    LABELED_SIZE, UNLABELED_SIZE, FULL_SIZE, multiplier
)
tokenized_dataset

Multiplier: 4


Map:   0%|          | 0/59924 [00:00<?, ? examples/s]

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

Map:   0%|          | 0/8262 [00:00<?, ? examples/s]

200 3000 3200 4
TRAIN (FOR only discriminator): 200
TRAIN (FOR GAN): 3800


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', 'labeled_mask'],
        num_rows: 3800
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', 'labeled_mask'],
        num_rows: 2500
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', 'labeled_mask'],
        num_rows: 8262
    })
    train_only_labeled: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', 'labeled_mask'],
        num_rows: 200
    })
})

<IPython.core.display.Javascript object>

### Train only discriminator

In [52]:
torch.cuda.empty_cache()
gc.collect()
from trainer import trainer as trainer_module
from trainer import gan_trainer as gan_trainer_module

trainer_module = imp.reload(trainer_module)
gan_trainer_module = imp.reload(gan_trainer_module)
from trainer import trainer

<IPython.core.display.Javascript object>

In [196]:
from copy import copy

BASE_CONFIG = copy(CONFIG)
BASE_CONFIG["GAN"] = False
BASE_CONFIG["gan_training"] = False
BASE_CONFIG["num_labels"] = NUM_LABELS
BASE_CONFIG["num_train_epochs"] = 5
BASE_CONFIG["LABELED_SIZE"] = LABELED_SIZE
# BASE_CONFIG

<IPython.core.display.Javascript object>

In [4]:
discriminator = model.DiscriminatorForTokenClassification(**BASE_CONFIG)

<IPython.core.display.Javascript object>

In [198]:
from trainer import trainer

trainer = imp.reload(trainer)

BASE_CONFIG["num_train_examples"] = len(train_only_labeled_dataloader.dataset)
trainer = trainer.TrainerTokenClassification(
    config=BASE_CONFIG,
    discriminator=discriminator,
    train_dataloader=train_only_labeled_dataloader,
    valid_dataloader=valid_dataloader,
    device=device,
)

Trainable layers 102


<IPython.core.display.Javascript object>

In [199]:
%%time
run = None
run = neptune.init_run(project=secret['neptune_project'], api_token=secret['neptune_token'], tags=['test']) 
run['config'] = trainer.config

for epoch_i in range(BASE_CONFIG['num_train_epochs']):
    print(f"======== Epoch {epoch_i + 1} / {BASE_CONFIG['num_train_epochs']} ========")
    train_info = trainer.train_epoch(log_env=run)
    valid_metrics = trainer.validation(log_env=run)
# run.stop()

https://app.neptune.ai/vmalperovich/gan-in-nlp/e/GAN2-306


You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


======== Epoch 1 / 5 ========
	Train loss discriminator: 2.180
	Test loss discriminator: 0.768
	Test accuracy discriminator: 0.866
	Test f1 discriminator: 0.000
======== Epoch 2 / 5 ========
	Train loss discriminator: 0.785
	Test loss discriminator: 0.694
	Test accuracy discriminator: 0.866
	Test f1 discriminator: 0.000
======== Epoch 3 / 5 ========
	Train loss discriminator: 0.678
	Test loss discriminator: 0.622
	Test accuracy discriminator: 0.866
	Test f1 discriminator: 0.000
======== Epoch 4 / 5 ========
	Train loss discriminator: 0.569
	Test loss discriminator: 0.529
	Test accuracy discriminator: 0.868
	Test f1 discriminator: 0.031
======== Epoch 5 / 5 ========
	Train loss discriminator: 0.459
	Test loss discriminator: 0.478
	Test accuracy discriminator: 0.872
	Test f1 discriminator: 0.083
CPU times: user 51 s, sys: 609 ms, total: 51.6 s
Wall time: 51.8 s


<IPython.core.display.Javascript object>

In [5]:
predict_info = trainer.predict(
    discriminator, test_dataloader, label_names=CONFIG["label_names"]
)
run["test"] = predict_info
run.stop()
predict_info

<IPython.core.display.Javascript object>

### Train via GAN

In [54]:
import gc

gc.collect()
torch.cuda.empty_cache()

<IPython.core.display.Javascript object>

In [202]:
from copy import copy

GAN_CONFIG = copy(CONFIG)
GAN_CONFIG["GAN"] = True
GAN_CONFIG["gan_training"] = True
GAN_CONFIG["GAN_TYPE"] = "attention"  # "attention" "mixed" "dummy"
GAN_CONFIG["mixed_fake_ratio"] = 0.2
GAN_CONFIG["LABELED_SIZE"] = LABELED_SIZE
GAN_CONFIG["UNLABELED_SIZE"] = UNLABELED_SIZE
GAN_CONFIG["num_train_epochs"] = 5

<IPython.core.display.Javascript object>

In [203]:
if GAN_CONFIG["GAN_TYPE"] == "dummy":
    generator = model.SimpleTokenGenerator(
        input_size=CONFIG["noise_size"],
        output_size=discriminator.encoder.config.hidden_size,
    )
elif GAN_CONFIG["GAN_TYPE"] == "attention":
    generator = model.ContextualTokenGenerator(
        input_size=GAN_CONFIG["noise_size"],
        output_size=discriminator.encoder.config.hidden_size,
    )
elif GAN_CONFIG["GAN_TYPE"] == "mixed":
    generator = model.ContextualTokenGenerator(
        input_size=GAN_CONFIG["noise_size"],
        output_size=discriminator.encoder.config.hidden_size,
        need_mixed_proj_layer=True,
    )
else:
    raise "ERROR"
discriminator = model.DiscriminatorForTokenClassification(**GAN_CONFIG)
generator

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Training with GAN mode on!
Default fake label index is -1


ContextualTokenGenerator(
  (attention): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=100, out_features=100, bias=True)
  )
  (out): Sequential(
    (0): Linear(in_features=100, out_features=768, bias=True)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Dropout(p=0.2, inplace=False)
    (3): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
)

<IPython.core.display.Javascript object>

In [204]:
from trainer import gan_trainer as gan_trainer_module

gan_trainer_module = imp.reload(gan_trainer_module)

GAN_CONFIG["num_train_examples"] = len(train_dataloader.dataset)
gan_trainer = gan_trainer_module.GANTrainerTokenClassification(
    config=GAN_CONFIG,
    discriminator=discriminator,
    generator=generator,
    train_dataloader=train_dataloader,
    valid_dataloader=valid_dataloader,
    device=device,
    save_path=CONFIG['save_path']
)


Trainable layers 102


<IPython.core.display.Javascript object>

In [205]:
%%time
run = None
tags = ['test']
run = neptune.init_run(project=secret['neptune_project'], api_token=secret['neptune_token'], tags=tags) 
run['config'] = gan_trainer.config


for epoch_i in range(GAN_CONFIG['num_train_epochs']):
    print(f"======== Epoch {epoch_i + 1} / {GAN_CONFIG['num_train_epochs']} ========")
    train_info = gan_trainer.train_epoch(log_env=run)
    result = gan_trainer.validation(log_env=run)
# run.stop()

https://app.neptune.ai/vmalperovich/gan-in-nlp/e/GAN2-307
======== Epoch 1 / 5 ========
	Train loss discriminator: 2.598
	Train loss generator: 0.672
Best model saved!
	Test loss discriminator: 0.365
	Test accuracy discriminator: 0.911
	Test f1 discriminator: 0.374
======== Epoch 2 / 5 ========
	Train loss discriminator: 0.897
	Train loss generator: 0.799
Best model saved!
	Test loss discriminator: 0.347
	Test accuracy discriminator: 0.929
	Test f1 discriminator: 0.532
======== Epoch 3 / 5 ========
	Train loss discriminator: 0.813
	Train loss generator: 0.783
Best model saved!
	Test loss discriminator: 0.316
	Test accuracy discriminator: 0.935
	Test f1 discriminator: 0.567
======== Epoch 4 / 5 ========
	Train loss discriminator: 0.771
	Train loss generator: 0.775
	Test loss discriminator: 0.361
	Test accuracy discriminator: 0.933
	Test f1 discriminator: 0.565
======== Epoch 5 / 5 ========
	Train loss discriminator: 0.758
	Train loss generator: 0.770
	Test loss discriminator: 0.417
	Tes

<IPython.core.display.Javascript object>

In [6]:
discriminator.load_state_dict(torch.load(CONFIG["save_path"]))
predict_info = gan_trainer.predict(
    discriminator, test_dataloader, label_names=CONFIG["label_names"]
)
run["test"] = predict_info
run.stop()
predict_info

<IPython.core.display.Javascript object>

# Experiments

In [40]:
from trainer import trainer as trainer_module
from trainer import gan_trainer as gan_trainer_module


# model_name = "distilbert-base-uncased"
# model_name = "bert-base-uncased"
# model_name = "google/electra-small-discriminator"
model_name = "google/electra-base-discriminator"
tokenizer = AutoTokenizer.from_pretrained(model_name)
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

<IPython.core.display.Javascript object>

In [41]:
CONFIG["encoder_name"] = model_name
CONFIG["noise_range"] = (0, 1)
CONFIG["noise_range_str"] = str(CONFIG["noise_range"])
CONFIG["noise_type"] = "normal"
CONFIG["num_train_epochs"] = 5
CONFIG["num_labels"]

37

<IPython.core.display.Javascript object>

In [42]:
tags = ["FINAL", "generator_type"]
NUM_TRIALS_GAN = 2

<IPython.core.display.Javascript object>

In [7]:
UNLABELED_SIZE = 3000
for LABELED_SIZE, per_label in tqdm_notebook(
    [(100, 10), (200, 20), (500, 50), (900, 100)]
):
    print(f"\n\n\n****************{LABELED_SIZE}***********\n\n\n")
    CONFIG["per_label_samples"] = per_label
    try:
        del discriminator
    except:
        pass
    torch.cuda.empty_cache()
    gc.collect()
    multiplier = int(np.log2(FULL_SIZE / LABELED_SIZE)) - 1
    multiplier = max(1, multiplier)
    tokenized_dataset = prepare_experiment_datasets(
        LABELED_SIZE, UNLABELED_SIZE, FULL_SIZE, multiplier
    )

    train_only_labeled_dataloader = DataLoader(
        tokenized_dataset["train_only_labeled"],
        batch_size=CONFIG["batch_size"],
        sampler=RandomSampler(tokenized_dataset["train_only_labeled"]),
        collate_fn=data_collator,
        pin_memory=True,
    )

    train_dataloader = DataLoader(
        tokenized_dataset["train"],
        batch_size=CONFIG["batch_size"],
        sampler=RandomSampler(tokenized_dataset["train"]),
        collate_fn=data_collator,
        pin_memory=True,
    )

    valid_dataloader = DataLoader(
        tokenized_dataset["validation"],
        batch_size=CONFIG["batch_size"],
        sampler=SequentialSampler(tokenized_dataset["validation"]),
        collate_fn=data_collator,
        pin_memory=True,
    )

    test_dataloader = DataLoader(
        tokenized_dataset["test"],
        batch_size=32,
        sampler=SequentialSampler(tokenized_dataset["test"]),
        collate_fn=data_collator,
        pin_memory=True,
    )
    for _ in range(1):
        # NO GAN
        print("NO GAN...")
        try:
            del discriminator
        except:
            pass
        torch.cuda.empty_cache()
        gc.collect()
        BASE_CONFIG = copy(CONFIG)
        BASE_CONFIG["num_train_examples"] = len(train_only_labeled_dataloader.dataset)
        BASE_CONFIG["GAN"] = False
        BASE_CONFIG["gan_training"] = False
        BASE_CONFIG["LABELED_SIZE"] = LABELED_SIZE
        discriminator = model.DiscriminatorForTokenClassification(**BASE_CONFIG)
        trainer = trainer_module.TrainerTokenClassification(
            config=BASE_CONFIG,
            discriminator=discriminator,
            train_dataloader=train_only_labeled_dataloader,
            valid_dataloader=valid_dataloader,
            device=device,
        )
        run = neptune.init_run(
            project=secret["neptune_project"],
            api_token=secret["neptune_token"],
            tags=tags,
        )
        run["config"] = trainer.config

        for epoch_i in range(BASE_CONFIG["num_train_epochs"]):
            print(f"== Epoch {epoch_i + 1} / {BASE_CONFIG['num_train_epochs']} ==")
            train_info = trainer.train_epoch(log_env=run)
            valid_metrics = trainer.validation(log_env=run)
        predict_info = trainer.predict(
            discriminator, test_dataloader, label_names=CONFIG["label_names"]
        )
        run["test"] = predict_info
        run.stop()

    for GAN_TYPE in ["dummy", "attention", "mixed"]:
        for _ in range(NUM_TRIALS_GAN):
            # GAN
            print("GAN...")
            del discriminator
            gc.collect()
            torch.cuda.empty_cache()
            GAN_CONFIG = copy(CONFIG)
            GAN_CONFIG["GAN"] = True
            GAN_CONFIG["gan_training"] = True
            GAN_CONFIG["GAN_TYPE"] = GAN_TYPE  # "attention" "mixed" "dummy"
            GAN_CONFIG["mixed_fake_ratio"] = 0.2
            GAN_CONFIG["LABELED_SIZE"] = LABELED_SIZE
            GAN_CONFIG["UNLABELED_SIZE"] = UNLABELED_SIZE
            discriminator = model.DiscriminatorForTokenClassification(**GAN_CONFIG)
            if GAN_CONFIG["GAN_TYPE"] == "dummy":
                generator = model.SimpleTokenGenerator(
                    input_size=CONFIG["noise_size"],
                    output_size=discriminator.encoder.config.hidden_size,
                )
            elif GAN_CONFIG["GAN_TYPE"] == "attention":
                generator = model.ContextualTokenGenerator(
                    input_size=GAN_CONFIG["noise_size"],
                    output_size=discriminator.encoder.config.hidden_size,
                )
            elif GAN_CONFIG["GAN_TYPE"] == "mixed":
                generator = model.ContextualTokenGenerator(
                    input_size=GAN_CONFIG["noise_size"],
                    output_size=discriminator.encoder.config.hidden_size,
                    need_mixed_proj_layer=True,
                )
            else:
                raise "ERROR"

            GAN_CONFIG["num_train_examples"] = len(train_dataloader.dataset)
            gan_trainer = gan_trainer_module.GANTrainerTokenClassification(
                config=GAN_CONFIG,
                discriminator=discriminator,
                generator=generator,
                train_dataloader=train_dataloader,
                valid_dataloader=valid_dataloader,
                device=device,
                save_path=CONFIG["save_path"],
            )
            run = neptune.init_run(
                project=secret["neptune_project"],
                api_token=secret["neptune_token"],
                tags=tags,
            )
            run["config"] = gan_trainer.config

            for epoch_i in range(GAN_CONFIG["num_train_epochs"]):
                print(f"== Epoch {epoch_i + 1} / {GAN_CONFIG['num_train_epochs']} ==")
                train_info = gan_trainer.train_epoch(log_env=run)
                result = gan_trainer.validation(log_env=run)
            discriminator.load_state_dict(torch.load(CONFIG["save_path"]))
            predict_info = gan_trainer.predict(
                discriminator, test_dataloader, label_names=CONFIG["label_names"]
            )
            run["test"] = predict_info
            run.stop()

<IPython.core.display.Javascript object>